# Resumen
Este cuaderno digital interactivo tiene como objetivo profundizar en el efecto de la heterogeneidad del dosel en los intercambios de calor, agua y carbono.

Para ello haremos uso del paquete de código abierto pyTSEB, el cual permite tanto simular flujos de energía, agua y CO<sub>2</sub> como estimar los mismos a partir de datos de teledetección (próxima o remota).

# Instrucciones
Lee con detenimiento todo el texto, y sigue sus instrucciones.

Una vez leida cada sección de texto ejecuta la celda de código siguiente (marcada como `In []`) presionando el icono de `Run`/`Ejecutar` o presionando en el teclado ALT + ENTER. Aparecerá una interfaz gráfica con la que poder realizar las tareas asignadas.

Como ejemplo ejectuta la siguiente celda para importar todas  las librerías necesarias para el correcto funcionamiento del cuaderno. Una vez ejecutada debería aparecer un mensaje de agradecimiento.

In [ ]:
%matplotlib inline
from ipywidgets import interact, interactive, fixed
from IPython.display import display
from functions import tseb_and_heterogeneous_canopies as fn
import numpy as np

# Ecuaciónes básicas
La evapotranspiración es básicamente un intercambio de energía y vapor de agua entre la superficie y la atmósfera. Es por ello que una variable fundamental para su cálculo es la estimación de la energía disponible de la superficie. 

![balance energético de la Tierra](./input/figures/radiation_balance.png "Balance energético de la Tierra").

Seǵun el estado hídrico de la superficie, esta energía disponible es utilizada tanto en evaporar el agua (la ET o $\lambda E$) como en aumentar su temperatura ($H$).

$$\lambda E + H = R_n - G$$

donde $G$ es el calor transmitido por conducción y almacenado en el suelo y $R_n$ es la radiación neta

La radiación neta ($R_n$) puede calcularse como:

\begin{equation}
R_n = S\downarrow \left(1 - \alpha\right) +  \epsilon \left(L\downarrow - \sigma LST^4\right)
\end{equation}

donde $S\downarrow$ y $L\downarrow$ son las radiaciones entrantes de onda corta y larga respectivamente y pueden obtenerse a través de estaciones o modelos meteorológicos, $\alpha$ y $\epsilon$ son el albedo y la emisividad del superficie, $LST$ es la temperatura de superficie, y $\sigma\approx5.67E-8$ (W/m²K⁴) es la constante de Boltzman. $\alpha$, $\epsilon$ y $LST$ pueden estimarse con mayor o menor grado de incertidumbre mediante datos y productos de teledetección.

Teniendo en cuenta la mayor magnitud de la radiación de onda corta ($S$) con respecto a la onda larga ($L$), en este cuaderno nos centraremos sobre todo a las propiedades del dosel vegetal que influyen en el albedo y en la fracción de radiación fotosintéticamente activa absorbida por la vegetación.

Por otro lado $G$ suele ser de menor magnitud, tendiendo a 0 cuando trabajamos en escalas diarias o más largas, o estimándose como una porcentaje de la radiación neta ($G \approx 0.1 R_n$ para supeficies vegetales densas; $G \approx 0.4 R_n$ para suelos desnudos) para escalas horarias o instantáneas.

El transporte efectivo de calor (o calor sensible, $H$) puede calcularse como:

$$H = \rho_{air} C_p \frac{T_0 - T_a}{r_a}$$

* $\rho_{air}$ y $C_p$ son la densidad del aire y su calor específico, y pueden considerarse prácticamente constantes.
* $T_0$ (K) es la temperatura aerodinámica de la superficie
> **NO CONFUNDIR CON LA TEMPERATURA DE LA SUPERFICIE QUE SE MIDE POR TELEDETECCIÓN**
* $T_a$ (K) es la temperatura del aire, medida a una altura sensiblemente superior a la altura del dosel ($z_T$)
* $r_a$ es la resistencia aerodinmámica (s/m), su recíproco sería la conductancia aerodinámica
> **NO CONFUDIR ESTA CONDUCTANCIA O RESISTENCIA AERODINÁMICA CON LA CONDUCTANCIA O RESISTENCIA ESTOMÁTICA**

Esta ecuación básica se puede interpretar de manera que el intercambio de calor entre la superficie y la atmósfera ($H$) viene determinado por el gradiente de temperaturas entre la superficie y las capa de aire por encima de la superficie. 

El transporte de calor es modulado por el factor de resistencia $r_a$ que engloba la habilidad mecánica y convectiva del aire para impedir o fomentar dicho calor:

$$r_{a}=\frac{\left[\ln\left(\frac{z_{T}-d}{z_{0H}}\right)+\Psi_{H}\right] \left[\ln\left(\frac{z_{U}-d}{z_{0M}}\right)+\Psi'_{M}\right]}{k^2\,U}$$

* $U$ es la velocidad del viento(m/s) medida a una altura $z_{U}$.
* $z_{T}$ es la altura (m) a la que se mide la temperatura del aire.
* $z_{U}$ es la altura (m) a la que se mide la velocidad del viento.
* $d$ y $z_{0M}$ son factores de rugosidad aerodinámica. Generalmente se puede simplificar el cálculo de $z_{0M}$ y $d_{0}$ en función de la altura de la vegetación ($h_c$)

$$z_{0M} = 0.125 h_c\\
d_{0} = 2/3 h_c$$

* $z_{0H}$ es la rugosidad aerodinámica ligada al transporte del calor. Suele estimarse como una fracción de $z_{0M}$

$$z_{0H} = 0.1 z_{0M}$$

* $k=0.41$ es la constante de von Kàrman

* $\Psi'_{H}$ y $\Psi'_{M}$ son funciones semiempíricas que modulan la resistencia y el perfil del viento en función de la estabilidad atmosférica.
> Si quieres conocer más detalles sobre su cálculo puedes ver el código en Python [para $\Psi'_{H}$](https://github.com/hectornieto/pyTSEB/blob/600664efd3e5ac4edab84e84fa5cb9d55c58c46f/pyTSEB/MO_similarity.py#L179)  y [para $\Psi'_{M}$](https://github.com/hectornieto/pyTSEB/blob/600664efd3e5ac4edab84e84fa5cb9d55c58c46f/pyTSEB/MO_similarity.py#L244) .

En las siguientes tareas vamos a intentar ir desgranando paso a paso todos y cada uno de los procesos que nos poermitirán más adelante poder calcular $r_a$ y entender su mecanismo.

# La radiación neta

## La ley de Beer-Lambert
La ley de Beer-Lambert es la ley física fundamental que permite describir e interpretar la mayoría de los procesos y fenómenos radiativos que nos interesan. Cuando un haz de luz atraviesa un medio que contiene un elemento absorbente de radiación, esta luz se atenúa de manera proporcional a la concentración del elemento ($\kappa$) y la longitud del camino que recorre la luz ($\ell$). La transmisión de luz o de radiación electromagnética:

\begin{equation}
T = e^{-\kappa\ell}
\end{equation}



Esta ley permite explicar por ejemplo la atenuación de la radiación solar desde la estratosfera hasta la superficie de la tierra: El aire está formado por gases y partículas que abosrben radiación (es más, la mayoría absorben sólo radiación en ciertas longides de onda, como es el ozono que abosrbe rayos UV). Cuanto más alto esté el sol en el cielo, el recorrido a través de la atmósfera es menor y por tanto la radiación solar que nos llega a la superficie será mayor.

![longitud atmosférica del sol](./input/figures/airmass.png "longitud atmosférica del sol").

### Aplicación a la Fracción de Radiación Interceptada (fIPAR)
La ley de Beer-Lambert nos permite también analizar la interceptación (y absorción) de radiación del dosel vegetal y equivalentemente la radiacion que finalmente llega al suelo.

Asumiendo que un dosel vegetal está formado en su mayoría por hojas, y que éstas interceptan y absorben radiación (sobre todo radiación fotosintéticamente activa, PAR), podemos ver cuánta radiación es inteceptada según la cantidad de hojas o la densidad foliar del dosel. Ésta suele definirse mediante el Índice de Área Foliar (LAI), la superficie de hojas por unidad de superficie de suelo. Cuanto mayor sea el LAI más probabilidades hay de que los rayos solares sean interceptados por la vegetación, y por tanto lleguen menos al suelo. De igual modo, el ángulo de elevación solar $\beta$, o su complemntario el ángulo zenital solar (SZA), juega también un papel importante, ya que cuanto más bajo esté el sol máyor camino tendrá que recorrer la luz a través del dosel y mayor será su atenuación. 

Como las hojas son elementos sólidos con un tamaño finito (al contrario que los gases atmosféricos), su orientación con respecto a la vertical también juega un rol importante, por lo que la ley de Beer-Lambert ha de modificarse para reflejar este fenómeno. En el siguiente gráfico puedes experimentar con este efecto. Puedes variar tanto la cantidad de hojas (en términos de LAI) como la orientación dominante de las hojas, desde hojas predominantemente verticales ($\approx$90º) a hojas predominantemente horizontales ($\approx$0º).

In [ ]:
w_fipar = interactive(fn.plot_fipar, lai=fn.w_lai, leaf_angle=fn.w_leaf_angle)
display(w_fipar)

Observa en el gráfico cómo las mayores atenuaciones de la luz (o lo que es lo mismo mayor fracción interceptada de radiación) ocurren siempre con ángulos de elevación solar menores, pero que esa tasa de atenuación depende en mayor medido de la cantidad de hojas y de su disposición angular.

El LAI depende en gran medida del desarrollo fenológico y del aumento de biomasa de la planta, mientras que su disposición angular suele ser específica de la especie, habiendo plantas con una disposición más erectófila (hojas predominantemente verticales) como pueden ser algunas gramíneas y plantas con una disposición más planófila (hojas predominantemente horizontales. Generalmente podemos asumir que especies erectófilas están más adaptadas a sobervivir en climas con altos niveles de insolación, con el fin de evitar condiciones extremas de irradiancia solar en las horas centrales del día, mientras que plantas planófilas tienen a desarrollarse en condiciones más de penumbra o de poca exposición, de manera que intercepten el máximo posible de radiación durante todo el día. 

Entre esos dos extremos la mayoría de las plantas presentan una diposición angular de las hojas más o menos aleatoria. Para estos casos el cálculo de la interceptación de radiación se simplifica como:

\begin{equation}
fIPAR = 1 - \exp \left(\frac{-0.5 LAI}{\cos\theta_s}\right)
\end{equation}
siendo $\theta_s$ el ángulo zenital solar.

> Si tienes interés en cómo realizar los cálculos para cualquier otro tipo de disposición angular de las hojas y tienes conocimientos de programación, picha [aquí](https://github.com/hectornieto/pyTSEB/blob/6dd5dffff08bc1f08edb3e89b4a79879e348146b/pyTSEB/TSEB.py#L1587).

## El albedo
Como hemos comentado anteriormente el albedo ($\alpha$) es la variable fundamental que determina la cantidad de radiación solar que es absorbida por la superficie. Se definie como la proporción de radiación de onda corta que es reflejada por la superficie. La radiación neta de onda corta es un balance entre la radación de onda corta incidente ($S\downarrow$) y la saliente o reflejada ($S\uparrow = \alpha S\downarrow$)

Las propiedades espectrales de la superficie terrestre son claves en la determinación de su albedo, por ejemplo la nieve tiene valores muy altos de albedo ($\approx 0.9$), y por tanto reflejan la mayoría de la radiación solar. Los océanos por el contrario tienen valores muy bajos de albedo ($\approx0.05$) por lo que aborben la mayoría de la energía solar que les llega. 

Las hojas, debido a la actividad fotosinténtica absorben una gran proporción luz por parte de las clorofilas y otros pigmentos, es por ello que también presentan valores relativamente bajos de albedo ($\approx0.15$). Por otro lado, el suelo según su composición mineralógica puede tener valores de albedo entre 0.20 y 0.40 o incluso más. Por tanto el albedo de un dosel vegetal dependerá sobre todo de la concentración de clorofilas de sus hojas pero también de la cantidad o densidad foliar del dosel, luego en menor medida del albedo del suelo, pero sólo en situaciones de escasa vegetación o estados fenológicos iniciales.

> Profundizaremos más sobre este tema en la práctica sobre el comportamiento espectral de la vegetación.

### Anisotropía de la vegetación
La mayoría de las superficies terrestres presentan cierta anisotropía a la hora de reflejar la radiación. Es decir, reflejan más radiación en unas direcciones que en otras. El caso más extremos son las superficies especulares (espejos y algunas láminas de agua y hielo) que reflejan la mayoría de la radiación en la dirección contraria a la posición del sol. El caso contrario son las superficies lambertianes, las  cuales dan la aparencia de reflejar por igual en todas las direcciones.

![Reflexión Lambertiana y especular](./input/figures/lambertian_and_specular_reflection.png "Reflexión Lambertiana y especular").

> La madera sin pulir es un ejemplo superficie lambertiana, mientras que la madera pulida y barnizada presenta por el contrario comportamientos especulares. 

La vegetación, al estar formada prinpicalmente de un cojunto más o menos agrupado de hojas, no se libra de este comportamieno anisotrópico, que provoca que según las geometría de iluminación, es decir según al posición del sol, refleja de manera distinta según la dirección, e incluso pueda variar el albedo global.

En el siguiente gráfico podrás ver este efecto. Se trata de un gráfico polar que simula el factor de reflectividad de una superficie vegetal para todas direcciones, tanto para la región del PAR como para todo el espectro solar. Los anillos concéntricos representan los distintos ángulos cenitales con respecto al nadir (0 para el punto central del gráfico y 90º para el anillo más externo). Las radiales del gráfico polar representa la dirección azimultal (0º hacia el Norte, 90º hacia el Este, 180º hacia el Sur y 270º hacia el Oeste). Tambien se muestra el valor integrado de la reflectividad, que reulta en el albedo.

La posición del sol está representada en el gráfico por una estrella. Por defecto al ejecutar el gráfico el sol está situado al sur con un ángulo cenital de 37 grados, lo que implica un día tipico de verano en el hemisferio Norte al mediodía.

In [ ]:
w_bidirectional = interactive(fn.bidirectional_reflectance,
                              cab=fn.w_cab, lai=fn.w_lai, leaf_angle=fn.w_leaf_angle, 
                              sza=fn.w_sza, saa=fn.w_saa, skyl=fn.w_skyl)
display(w_bidirectional)                             

* Modifica sustancialmente la concentración de clorofila. ¿Cómo varía la reflectividad y el albedo tanto en el PAR como en el espectro completo solar?
* Modifica la posición del sol. Por un lado podrás ver que siempre hay un pico de reflectividad alrededor de la posición solar. Esto se trata del la zona de la vegetación que está más illuminada por el sol. Por el contrario, en la dirección opuesta se ecuentran los valores de reflectividad menores, al estar las hojas más ocluidas de los rayos de sol. Además los valores de albedo también varían según la posición del sol, es decir, según la hora del día.
* Esta variación es más significativa cuanta menor sea la proporción de radiación difusa. Aumenta la proporción de radación difusa, simulando días con mayor cobertura nubosa, verás que la posición del sol, como es de esperar ya no es relevante.

## Radiación neta de onda corta
Teniendo en cuenta lo visto hasta hora, el LAI, la distribución angular de las hojas la concentración de colorofila son las variables fundamentales que determinan la absorción de radiación del cultivo. 

Hasta ahora todas las simulaciones y procesos que hemos realizado asumen un dosel vegetal o un cultivo horizontalmente homogéneo, o al menos suficientemente denso. Sin embargo, ¿qué ocurre cuando trabajamos con cultivos en hilera, como pueden ser viñedos o explotaciones intensivas de frutales, entre otros?

La cruda realidad es que todo restula algo más complejo ya que por un lado el dosel vegetal está agrupado en torno a un tronco o tallo, dejando partes del suelo completamente expuestas. Por otro lado, la dimensión y orientación de la hilera también afecta a cómo la luz es interceptada y transmitida entre el cultivo y el suelo.

Para evaluar esta situación es necesario considerar parámetros estructurales del cultivo adicionales:

* La altura del dosel en metros, ya que doseles más altos generarán sombras en el suelo más alargadas entre las hileras.
* El porcentaje relativo de ocupación del mismo con respecto a la separación entre hileras, que indica la anchura del dosel con respecto al espacio entre hileras.
* La orientación de las hileras con respecto al Norte. 0 indica que la hilera está orientada de Sur a Norte, valores negativos indica que la hilera va de NW as SW, valores positivos del NE and SW, con -90 y/o 90 indicando orientación E-W.

En el siguiente gráfico podremos ver la evolución de la radiación solar neta de un cultivo herbáceo y su comparación con un cultivo heterogéneo a lo largo un día típico de verano para una latitud dada (independientemente de que sea el hemisferio Sur o Norte). En ambos se asumen las mismas condiciones espectrales de hoja y suelo (p.ej. $C_{a+b}\approx40\mu g/cm^2$}. Este gráfico asume que las condiciones de nubosidad permanecen constantes durante el día, con una proporción de radiación difusa fija. Si las condiciones de nubosidad fueran cambiantes la curva presentaría una forma más aserrada, formando valles y crestas en los momentos en los que el sol estuviese visible u oculto tras las nubes.

El gráfico muestra el porcentaje de radiación de onda corta absorbida por la superficie (curva de color negro, $S_n$) así como el porcentaje de radiación absorbido por el cultivo (en verde, $S_{n,C}$). Cuando el sol está por debajo del horizonte (antes del amanecer y después del ocaso) no se muestran datos.

In [ ]:
w_sn = interactive(fn.plot_net_solar_radiation,
                   lai=fn.w_lai, leaf_angle=fn.w_leaf_angle, h_c=fn.w_hc, f_c=fn.w_fc,
                   row_distance=fn.w_interrow, row_direction=fn.w_psi, skyl=fn.w_skyl, 
                   fvis=fixed(0.55), lat=fn.w_lat)
display(w_sn)

Observa que al forzar la clorofila constante, las variaciones de radiación neta son apenas perceptibles con variaciones de LAI. Sin embargo el porcentaje de radiación absorbida por la vegetación varía considerablemente con el LAI. Esto tiene implicaciones importantes ya que el LAI influye en el reparto de energía entre suelo y vegetación y por lo tanto en la capacidad de la superficie de evaporar el agua del suelo y/o la capacidad transpirativa y fotosintética del cultivo.

Mantén valores relativavente bajos de LAI (<1.5) y observa de nuevo el efecto de la distribución angular de las hojas en el dosel. Hojas más verticales (`Leaf Angle`\approx$90) provocarían una disminución de la radiación absorbida por la vegetación en las horas centrales del día, cuando el sol está más elevado y la irradiancia solar es mayor. Observa que este efecto sin embargo ya no ocurre cuando hay una mayor proporción de radiacion difusa. En ese caso casi toda la radiaicón viene indistintamente de todas direcciones y por tanto la distribución angular de la hoja no juega un papel importante.

Teniendo en cuenta que el sol sale hacia el Este, alcanza su cénit al Norte o al Sur seǵun el hemisferio, y se pone hacia el Oeste, observa el efecto que tiene cambiar la orientación de las hileras de N-S a E-W. El mínimo de radiacion interceptada por el cultivo es cuando el sol está en paralalo a la dirección de la hilera.

Observa que cuanta mayor sea el porcentaje de ocupación del cultivo con respecto al ancho de la hilera, el cultivo se comporta de manera más similar a un cultivo homogéneo (sección previa).

Hay un efecto combinado entre la altura del cultivo y la separación entre hileras. Hileras estrechas en cultivos bajos tienen un efecto similar que hileras anchas con vegetación alta.

Finalmente, al igual de lo que hemos estado observando en los gráficos anteriores, estos efectos se disipan cuanta mayor sea la radiación difusa, es decir cuanto mayor sea la nubosidad del cielo.

# El perfil logarítmico del viento
La variación de la velocidad del viento con respecto a la altura se puede calcular según un perfil logarítmico. Ignorando los efectos de turbulencia convectiva y por tanto asumiendo una estabilidad neutra:

$$u\left(z\right) =\frac{u_*}{k}\left[\log\left(\frac{z - d}{z_{0M}}\right)\right]$$

Esta atenuación logarítmica de viento está íntimamente relacionada con la resitencia aerodinámica. 

La atenuación del viento será más fuerte cuanto mayor sea la rugosidad de la superficie, expresados por los parámetros $z_{0M}$ y $d_{0}$. 

$u_{*}$ es la velocidad de fricción. De nuevo, simplificando e ignorando de momentos los efectos de la estabilidad atmosférica $u_{*}$ se puede calcular como:

$$u_* = \frac{k\,U} {\log\left(\frac{z_u - d_0}{z_{0M}}\right))}$$

con $U$ es la velocidad del viento medida a la altura $z_U$.

Por tanto, podemos extrapolar este perfil del viento para evaluar la velocidad del viento justo por encima del dosel vegetal, el cual tiene una altura $h_c$:

$$u_C =\frac{u_*}{k}\left[\log\left(\frac{h_c - d}{z_{0M}}\right)\right]$$

## Perfil del viento en un dosel heterogéneo
Para comprender mejor el efecto del follaje en la atenuación del viento entre el dosel y el suelo podemos ver mejor cómo el viento se atenúna en un dosel heterogéneo. La gran mayoría de tipos de vegetación, y en particular plantas leñosas, presentan una densidad foliar que varía en altura. Las plantas suelen tener un tallo o un tronco y las hojas por ejemplo empiezan a crecer a una determinada algura, presentando una copa más o menos globosa.

En esta siguiente práctica vamos simular qué pasas en estos tipos de vegetación, aplicando un perfil de viento dentro del dosel más realista (también por desgracia más complejo). Aparte del LAI Y la altura del dosel que ya habíamos utilizado antes, podemos variar un nuevo parámetro, la altura relativa con respecto a la altura total del dosel a partir de la cual comienza el follaje. Por defecto este valor está en 0.5, lo que significa que él inicio de copa comienza a la mitad del tronco o tallo. Para no complicar mucho más la parametrización, asumimos además que la mayor densidad de hojas se encuentra a en el punto medio de la copa (es decir, la copa tiene una forma más o menos esférica)

En la siguente celda interactiva podrás ver la atenuación del viento, tanto sobre el dosel como dentro del dosel, en función de la altura de la vegetación y de la densidad foliar. Para esta gráfica se mantiene una velocidad del viento de $U=5$ m/s medida 10m sobre el terreno ($z_u=10$), y como se ha mencionado los cálculos se han realizado asumiendo condiciones de estabilidad neutras.

El gráfico muestra la velocidad del viento justo a la altura del dosel mediante una estrella. Por debajo de la misma el perfil del viento, al encontrarse con el follaje, se atenúa de una forma completamente distinta a como se atenúa sobre el dosel.

Cuando hagas variar la altura de vegetación o el LAI, verás que se mantiene un perfil de referencia. Este sería el perfil del viento para el cultivo de referencia de la ET de FAO. El cual se define por una altura de la vegetación  de 0.12 m y un $LAI\approx3$. Por otro lado se muestra asímismo el perfil del viento que tendría un cultivo homogéneo.

Compara cómo de diferente puede ser el perfil del viento con respecto al perfil de referencia de FAO. En particular aumenta significativamente la altura de la vegetación. Verás que la velocidad de viento decrece mucho más rápido con la altura. Así verás además mejor cómo el viento se atenúa a través de un dosel.

Cuanto más rápido varíe la velocidad el viento con la altura se producirá un transporte más turbulento entre la superficie y la atmósfera. Esto genererará más remolinos y un transporte de calor y vapor de agua más eficiente, ya que los remolinos expulsan hacia la atmósfera las parcelas de aire que están en contacto con la superficie.

> [En este vídeo que muestra los remolinos (eddies) que se producen al aumentar la rugosidad de la superficie. En este caso se trata de un ala de aeroplano.](https://youtu.be/5YwnY0wPphA)

Por otro lado, cuanto mayor sea la densidad de hojas, expresada por el índice de área foliar (LAI) la atenuación del viento será mayor dentro del dosel, produciéndose igualmente intercambios de calor y vapor de agua más eficientes entre el cultivo y el suelo.

In [ ]:
w_wind = interactive(fn.wind_profile_heterogeneous, lai=fn.w_lai, zol=fixed(0), h_c=fn.w_hc, 
                     hb_ratio=fn.w_hb_ratio, h_c_max=fn.w_h_c_max)
display(w_wind)

Observa cómo de diferente es la atenuación del viento en este tipo de doseles con respecto al gráfico de la tarea anterior (hojas uniformemente distribuidas a lo largo del dosel).

> En este y en todos los gráficos interactivos, si mofificas el valor de una variable automáticamente se actualiza en el resto de los gráficos y cálculos.

Puedes ver que la mayor atenuación del viento ocurre en torno a la altura con mayor densidad de hojas. Esto es debido a que el viento en ese momento se encuentra con mucho más obstáculos y por tanto pierde fuerza y velocidad. En cambio, en las zonas más cercanas al suelo, donde ya apenas hay biomasa foliar el viento se atenúa con  mucha menor fuerza.

## Efecto en la resistencia aerodinámica

Como ya hemos comentado, la velocidad del viento y cómo éste se atenua tiene un efecto importante en cómo de eficiente es el transporte de calor y vapor de agua desde las distintas superficies (hojas y suelo) hacia la atmósfera, modificándose por tanto las resistencias al transporte de calor y vapor de agua.

En la siguiente tarea podrás ver cómo la resistencia aerodinámica varía con la rugosidad de la superficie (o con la altura del dosel) y con la velocidad del viento. 

### La estabilidad atmosférica y la turbulencia convectiva.
Hemos visto hasta ahora que la resistencia al transporte de calor y vapor de agua decrece (es decir el transporte se hace más efectivo) cuando aumenta la velocidad del viento ($U$) y cuando aumenta la rugosidad aerodinámica de la superficie ($z_{0M}$ y $d$). Esta es la parte mecánica 

Ahora veremos que cuando aumenta la inestabilidad atmosférica por fenómenos convectivos se generará aún más turbulencia y por tanto el trasporte de calor se hace también más eficiente. 

La estabilidad puede expresarse en términos relativos entre la rugosidad aerodinámica y la longitud de Monin-Obukhov $z_{0m}$ y $L$ ($\xi=z_{0m}/L$), la cual depende principalmente del calor sensible de la superficie ($H$)

> Si quieres ver más detalles sobre su cálculo puedes pinchar [aquí](https://github.com/hectornieto/pyTSEB/blob/600664efd3e5ac4edab84e84fa5cb9d55c58c46f/pyTSEB/MO_similarity.py#L134 "https://github.com/hectornieto/pyTSEB/blob/600664efd3e5ac4edab84e84fa5cb9d55c58c46f/pyTSEB/MO_similarity.py#L134")

Cuanto más negativo sea el coeficiente de estabilidad, mayor será la inestabilidad atmosférica y por tanto mayor eficiente será el transporte de calor y vapor de agua debido a la convección del calor.

> Recordemos que el aire caliente pesa menos que el aire frío por lo que una superficie con una temperatura mayor que la del aire calentará las capas inferiores del mismo, generándose un transporte de calor del aire cálido y ligero situado en las capas inferiores hacia arriba. De manera recíproca el aire de las capas superiores que no se ha calentado aún por las mayores temperaturas de la superficie pesarán más y tenderán a desplazarse hacia abajo.
> Puedes ver un video sobre este fenómeno [aquí](https://youtu.be/OM0l2YPVMf8).

### Effecto  de la velocidad del viento, la rugosidad y la estabilidad en la resistencia aerodinámica y el transporte de calor
Vamos a ver ahora el efecto que tiene la inestabilidad atmosférica en la resitencia aerodinámica. En esta tarea hemos añadio una nueva variable a nuestro cálculo, el factor de estabilidad $\xi$. En condiciones normales durante el día, la superficie va absorbiendo calor y por tanto durante el día suele haber condiciones de inestabilidad atmosférica, eso se traduce en que el parámetro de estabilidad suele ser igual o menor que cero (igual indicando una atmósfera neutra). Por la noche en cambio, la superficie terrestre puede estar más fría que el aire, y por tanto se producen fenómenos de inversión térmica y estabilidad atmosférica. 

> En balance, durante un día completo la atmósfera tiende siempre a la inestabilidad, lo cual en parte permite la circulación general de la atmósfera en el planeta

In [ ]:
w_ra = interactive(fn.plot_aerodynamic_resistance, zol=fn.w_zol, h_c=fn.w_hc)
display(w_ra)

Como habrás visto cuanto mayor la resistencia aerodinámica disminuye con la velocidad del viento, pero también disminuye con la rugosidad de la superficie. Esto hace que el transporte de calor sea más eficiente, lo que conlleva que se disipe mejor el exceso de calor y el vapor de agua de la vegetación.

Ello tiene implicaciones cruciales a la hora de interpretar el gradiente entre la temperatura de la superficie (que podremos estimar con teledetección) y la temperatura del aire. 

> Un mismo gradiente de temperaturas puede indicar estŕes hídrico o no, según la rugosidad de la superficie y las condiciones meteorológicas.

Haz variar el coeficiente de estabilidad y compara las distintas gráficas. Observa cómo los valores de resistencia aerodinámica se reducen, para una misma velocidad del viento y altura de la vegetación, con la inestabilidad atmósferica. Por el contrario, cuando hay condiciones de inversión térmica y estabilidad atmosférica  (valores positivos de `Estabilidad`), el transporte de calor se dificulta, lo cual se refleja en un aumento significativo de la resistencia aerodinámica.

# La Evapotranspiración y el Flujo de Calor
En esta segunda parte de esta práctica vamos a integrar todo lo que hemos visto hasta ahora para ver su efecto en el evapotranspiración y en la temperatura de la superficie. Para ello haremos uso de todos los cálculos descritos ateriormente y los modelos de Penman-Monteith y Shuttleworth-Wallace.

El modelo de Penman-Moteith es probablemente el modelo más popular para evaluar los flujos de calor y vapor de agua entre la superficie y la atmósfera. De hecho la ET de referencia calculada por el método propuesto por FAO se basa en este modelo. Se trata de un modelo de una capa tipo "big leaf", en el que se engloban los procesos de intercambio de calor y humedad del suelo y vegetación en un único sistema:

![Penman-Monteith](./input/figures/penman_monteith_model.png "Esquema de resistencias del modelo de Penman-Monteith")

El flujo de calor latente o evapotranspiración se calcula entonces como:

$$\lambda E = \frac{\Delta\left(R_n - G\right) + \rho_{air}C_p VPD/r_a}{\Delta + \gamma\left(1 + R_c/r_a\right)}$$

* $\Delta=2504\exp\left(\frac{17,27T_{air}}{T_{air}+237,2}\right)$ es la pendiente de la saturación de vapor, con T_{air} expresado en ºC.
* $\gamma$ es la constante psicrométrica $\approx$ 0.668 mb/ºC.
* $VPD=e_s - e_a$ es el déficit de presión de vapor (mb).
* $R_c$ es la resistencia al tranporte de vapor de agua de la superficie.

$R_c$ está relacionada con la resistencia estomática al tranporte de vapor de agua ($r_{st}$), de modo que:

$$ R_c = \frac{r_{st}}{LAI_{eff}}$$

donde $LAI_{eff}$ es el $LAI$ que está transpirando efficientemente. 

La ET de referencia de FAO asume que sólo la mitad superior del dosel transpira. Teniendo en cuenta que el cultivo de referencia de FAO es un césped de 0.12 m de altura con un LAI total de 2.88 por lo que $LAI_{eff}=1.44$. Como ese cultivo de referencia está bajo condiciones bien regadas, tiene una resistencia estomática máxima de 100 s/m, por lo que finalmente para FAO56 $R_c=70$ s/m.

> Para más información sobre la ET de referencia de FAO pincha [aquí](./lecturas_adicionales/FAO56.pdf).

La principal ventaja de Penman-Monteith es su simplicidad (por ello es el modelo usado por la ET de referencia de FAO), mientras que su principal desventaja es que no puede modelar separado los flujos de calor y agua del suelo (evaporación) y de la planta (transpiración). Esto hace también que presente dificultades a la hora de modela situaciones de vegetación poco densa o cultivos en hilera.

Por otro lado el modelo de Shuttleworth-Wallace es un modelo de dos capas, en la que separa los flujos entre el suelo y la vegetación, permitiendo la simulación por separado de la evaporación del suelo y la transpiración de la planta. Asímismo permite evaluar las interacciones e intercambios de calor suelo/planta, siendo más fiable a la hora de estimar los flujos en cultivos complejos. Sin embargo presenta una mayor complejidad de cálculo siendo generalmente recomendable usar algún tipo de software o código.

![Shuttleworth-Wallace](./input/figures/shuttleworth_wallace_model.png "Esquema de resistencias del modelo de Shuttleworth-Wallace")

Como variables adicionales requeridas por Shuttleworth-Wallace es la resistencia al tranporte de vapor de agua del suelo ($R_{ss}$) la cual depende de la humedad superficial del suelo.

## Efecto de la humedad superficial del suelo en la evapotranspiración
Como ejemplo del interés adicional del modelo Shuttleworth-Wallace nos proporciona frente a Penman-Monteith, en la siguiente gráfica podrás ver cómo la ET diaria varía con distintos niveles de humedad del suelo, expresados en términos de resistencia del suelo al tranporte de vapor de agua ($R_{ss}$).

La siguente tarea consideran unas condiciones meteorológicas fijas:
* Irradiancia solar diaria de 300 W/m².
* Temperatura media diara de 25ºC (298K)
* Humedad relativa del 50% ($VPD\approx$ 15mb) 
* Velocidad del viento a 10m sobre el suelo de 5 m/s


## Efecto de la conductancia estomática
La resistencia estomática representa el estado hídrico del cultivo. A partir de ahora, en lugar de hablar de resistencia estomática hablaremos de su recíproco, la conductancia estomática ($g_s = 1 / r_{st}$) ya que esta es probablemnte la variable más usada desde el punto de vista de fisiología vegetal. Además mucha de la instrumentación para medir conductancia estomática presenta unidades de mmol/m²s¹. 

La conductancia estomática valores mínimos en torno a 0.4-0.5 mmol/m²s¹ cuando el cultivo está bien hidradato

> La resitencia estomática del cultivo de referencia de FAO es de 100 s/m, lo que es igual a una conductancia estomática 0.415 mmol/m²s¹ (condciones estándar de 25ºC y a nivel del mar).

Cuando empieza a haber déficit de humedad en la zona radicular del suelo, la respuesta fisiológica más comun en las  plantas es cierra parcial o totalmente los estomas. con el fin de evitar pérdidas significativas de agua y conservar parte del agua en el suelo. Esto por otro lado tiene la consencuencia de una menor fijación de CO$_2$ y por tanto una reducción en la productividad de la planta. Este cierre estomático supone una reducción (incremento) de la conductancia (resistencia) estomática.


## Micro-advección. Interacción de flujos suelo-vegetación
En anotaciones previas se ha comentado que se puede producir un calentamiento de la vegetación cuando aumenta la resistencia del suelo y viceversa, el suelo puede calentarse si disminuye la conductancia estomática. Este hecho puede parecer contraintuitivo, pero en realidad es la consecuencia de un efecto que recoge el modelo Shuttelworth-Wallace como es la interacción de flujos entre suelo y vegetación.

Recordemos que todo transporte de calor y vapor de agua se produce mediante la turbulencia y torbellinos del aire. Por lo tanto si la superficie del suelo está seca y recibe suficiente radiación, éste se calentará, fomentando una advección de aire caliente y seco hacia las capas de aire superiores donde se encuentra el follaje. Esto podría fomentar un mayor déficit de presión de vapor potenciando la transpiración de la planta, o en su defecto aumentando su temperatura. De manera inversa, un dosel denso suficientemente irrigado aporta aire fresco y húmedo al entorno del suelo, por lo que éste podría tener una temperatura inferior a la esperada.

In [ ]:
w_et = interactive(fn.fluxes_and_resistances, r_ss=fn.w_r_ss, g_st=fn.w_g_st, h_c=fn.w_hc)
display(w_et)

Cuanto mayor sea $R_{ss}$ más seco está la superficie del suelo, con un límite inferor de 0 s/m para un suelo completamente inundado.

> $R_{ss}$ representa la humedad superficial del suelo, en su capa estrictamente más superficial. Puede darse el caso, más habitual de lo que uno puede pensar. de tener un suelo superficialmente seco ($R_{ss} >> 1000$ s/m) y aún así que la planta esté bien hidratada ($r_{st}$ o $R_c$ <= 100 s/m), ya que ésta obtiene la humedad de capas más profundas en el suelo.

Observa en el panel superior cómo varía la ET total de Shuttleworth-Wallace (línea azul) frente al modelo de Penman-Monteith que no considera este fenómeno. El eje x del muestra cómo el LAI efectivo incrementa la ET, ya que la resistencia de la superficie dismuye con el LAI ($R_c = r_{st}/LAI_{eff}$). Además se representa con una estrella la ET que resultaría al cacular la ET de referencia de FAO (recuerda que el LAI_eff de FAO56 es de $\approx$1.44). 

Presta sobre todo atención a los valores bajos de LAI y de LAI=0 (suelo desnudo) cuando $R_{ss}$ es cero (suelo inundado) o cercano a cero. Penman-Monteith al ser un modelo "big leaf" y no incorporar esta variable no es capaz de representar la evaporación directa del agua en el suelo.

Por otro lado, con un valor de $R_{ss}$ en torno a 5000 s/m las curvas son ya muy parecidas y coincidentes en el valor de ET referencia de FAO:

El panel intermedio muestra información adicional del modelo de Shuttleworth-Wallace, que es la fracción de ET que es usada en la tranpiración (linea verde) y en la evaporación del suelo (linea amarilla). Igualmente, cuanto mayor sea el LAI, más radiación interceptará y absorberá la planta (menos llegará al suelo) y por tanto la tranpiración dominará sobre la evaporación del suelo.

Finalmente el panel inferior muestra la diferencia de temperatura de la vegetación (línea verde), suelo (línea amarilla) con respecto a la del aire, simuladas por Shuttleworth-Wallace. Además se muestra la diferencia de temperatura aerodinmámica con respecto a la del aire simulada por Penman-Monteith. 

Observa en este panel cómo las temperaturas decrecen con el LAI. Un dosel denso disipa mucho mejor el calor que un dosel poco desarrollado, para idénticas condiciones hídricas. 

Mira también como la temperaturas de suelo y la temperatura aerodinámica aumentan con el incremento de $R_{ss}$. 

> Puedes incluso percibir que la temperatura de la hoja varía ligeramente con la resistencia del suelo, sobre todo en valores bajos de LAI. Esto  a priori puede resultar poco  lógico, pero ya explicaremos este fenómeno en una tarea más adelante.

Observa ahora cómo varía la ET según vamos reduciendo la condunctancia estomática, y por tanto simulando condiciones de estrés hídrico del cultivo.

> Para esta parte puedes dejar fijado un valor de $R_{ss}=5000$ s/m para el modelo Shuttleworth-Wallace. 

Observa que según vamos reducionendo la conductancia estomática, ambos modelos de Penman-Monteith como de Shuttleworth-Wallace se desvian significativamente del valor de ET de referencia de FAO (condiciones óptimas de humedad del suelo)

Además, cuanto más estresada esté la planta (menor conductancia estomática) menor transpiración con respecto a la evaporación del suelo. Siendo el caso extremo cuando $g_s=0$ mmol/m²s¹ donde la transpiración es cero  y sólo hay una mínima evaporación del suelo.

Observa a su vez que cuando menor sea la conductancia estomática mayor será la temperatura de la vegetación y la temperatura aerodinámica.

> Puedes incluso percibir que la temperatura del suelo varía ligeramente con la conductancia estomática, más perceptibles en valores altos de LAI. Esto a priori puede resultar poco lógico, pero en el siguiente ejercicion explicaremos este fenómeno.

Experimenta ahora dejando fijo un valor alto de conductancia estomática con valores altos de resistencia del suelo para despues reducir la resistencia del suelo a valores cercanos a 0. Verás como a valores bajos de LAI la temperatura de la vegetación varía en cierto punto, a pesar de tener una conductancia alta y constante. 

Repite este ejercicio pero manteniendo ahora un valor alto de resistencia en el suelo conductancia con valores altos y bajos de conductancia estomática.  Verás como a valores altos de LAI la temperatura del suelo varía en cierto punto, a pesar de tener una resistencia del suelo constante.

## Efecto de la altura de la vegetación

Ya hemos visto anteriormente el efecto que tiene la altura de la vegetacion (a través de su relación con la rugosidad aerodinámica) en el cálculo de la resistencia aerodinámica. Ahora veremos este efecto directo en la evapotranspiración y la temperatura.

Para comenzar, restaura los valores de conductancia estomática que usa por defecto la ET de referencia de FAO ($g_s=0.41$ mmol/m²s¹). También deja fijo la resistenciaa del suelo a un valor en torno a 5000 s/m. Ahora haz variar la altura de la vegetación, desde un valor inicial de 0.12 que es la altura del cultivo de referencia a otras alturas. Verás que según aumenta la altura de la vegetación, y por tanto aumenta su rugosidad aerodinámica, la evapotranspiración aumenta considerablemente (¡puede llegar en algunos casos a salirse del rango de valores del gráfico!)

# Sobre la temperatura de superficie

Como habíamos mencionado al principio es imperativo tener en cuenta que la temperatura aerodinámica ($T_0$), que es la temperatura a partir de la cual se puede calcular directamente el flujo de calor sensible, es distinta a la temperatura de superficie (LST) que podemos medir mediante sensores térmicos de teledetección. Por ello es necesario ser consciente de esa difernecia y cómo interpretar la temperatura de superficie a la hora de estimar correctamente la ET real por teledetección.

> De hecho los distintos modelos de ET por teledetección difieren principalmente en cómo utilizan la temperatura de superficie a la hora de derivar $H$ y $\lambda E$.

En primer lugar, debido a que el calor se transfiere de manera menos eficiente con el movimiento del aire, T_0 y LST es necesario definir correctamente la rugosidad aerodinámica al transporte de calor ($z_0H$), y la relacion $z_{0H} = 0.1 z_{0M}$ puede no ser valida para todo tipo de superficies.

En segundo lugar, y mucho más importante, es el hecho que según el ángulo de observación del sensor la temperatura medida puede variar significativamente. Un sensor de teledetección puede observar un punto de la superficie desde una observación próxima al nadir (Ángulo cenital de observación, $VZA\approx0$) hasta con 50º o 60º de inclinación con respecto al nadir.

> Ya veremos más adelante en detalle cómo los sensores de teledetección capturan la señal de la superficie.

Esto tiene como consecuencia de que según el ángulo zenital de observación (VZA) el sensor puede estar viendo una mayor o menor proporción de vegetación. En efecto, en la práctica del día anterior habíamos visto que según el ángulo de iluminación solar, y según la densidad y disposición de las hojas, los rayos solares son interceptados en mayor o menor medida. Este efecto es perfectamente recíproco cuando se trata con la geometría de observación en lugar de la geometría del sol.

Esta fracción de vegetacion observada también se ve afectada por la distribución angular de las hojas, tal y como se muestra en las curvas comparativas del gráfico. La línea negra punteada muestra la fracción observada por un dosel con una distribución típica (ángulo típico de 57º) a modo comparativo. Observa  por ejemplo la poca fraccion de vegetación observada si el sensor estuviese mirando justo hacia el nadir (VZA=0) para un dosel con una distribución predominantemente vertical de las hojas (Leaf Angle=90)

Como el suelo y la vegetación suelen presentar temperaturas distintas, según la geometría de obsevación y por tanto según la fracción de vegetación y suelo observada por el sensor, éste medirá una temperatura de superficie distinta.

Esto puedes verlo en esta última gráfica. En función a las temperaturas de suelo y vegetación simuladas en la tarea  [La Evapotranspiración y el Flujo de Calor](#La-Evapotranspiración-y-el-Flujo-de-Calor), puedes definir la geometría de observación y la diposición angular de las hojas para ver cómo cambia la temperatura de superficie medida.

> Puedes modificar los valores de la simulación desde [aquí](#Efecto-de-la-altura-de-la-vegetación) y automáticamente se reflejarán los cambios.

Además, se presentan dos métodos de cálculo de la temperatura de suerficie:
* un método simplificado que simplemente es una media de las temperaturas de hoja y suelo ponderada por la fracción de vegetación observada ($f_c\left(\theta\right)$):
$$LST^4=f_c\left(\theta\right)T_{C}^4+\left[1-f_{c}\left(\theta\right)\right]T_{S}^4$$
* un método analítico mucho más complejo que tiene en cuenta fenómenos de emisión de energía y emisividad tanto de la hoja como del suelo.

In [ ]:
w_fveg = interactive(fn.get_land_surface_temperature,
                     vza=fn.w_vza, leaf_angle=fn.w_leaf_angle, temperatures=fixed(w_et), 
                     e_v=fn.w_ev, e_s=fn.w_es)
display(w_fveg)

# Aplicación en una situación real

Para poner en práctica todo lo aprendido hasta ahora, vamos a utilizar datos reales en un olivar de la provincia de Jaén. Se trata de una plantación tradicional de olivar con una cobertura aproximada de 20% y copas de 4m de altura. Los datos meteorológicos, de temperatura de superficie y de evapotranspiración se encuentran en [./input/meteo/meteo_olive.csv](./input/meteo/meteo_olive.csv), los cuales han sido extraidos del portal ICOS. Para más información sobre la parcela de estudio y consultar los datos completos pincha [aquí](https://meta.icos-cp.eu/objects/Sqh69qK2l4QpSBo3U5ZAqWFa).

Por otro lado, también se ha extraido para esa parcela la evolución del LAI, estimada con datos VIIRS (300m de resolución espacial). Estos datos pueden encontrarse en [./input/canopy/LAI_FAPAR_ES-Cnd.csv](./input/canopy/LAI_FAPAR_ES-Cnd.csv).

En primer lugar puedes explorar las series temporales de flujo latente ($\lambda E$, o ET en términos de energía), temperatura radiométrica y LAI.




In [ ]:
w_ts = interactive(fn.create_flux_timeseries,
                   df=fixed(fn.METEO_DATA), daily_df=fixed(fn.METEO_DAILY_DATA),
                   date_range=fn.w_dates, include_obs=fixed(False))
display(w_ts)

## Simulación de ET y temperaturas basado en índice de estrés
Podemos utilizar el modelo de Shuttleworht & Wallace mencionado anteriormente, para simular cuál sería la transpiración y la temperatura de superficie de un cultivo heterogéneo sometido a las mismas condiciones ambientales que la parcela de olivo. 

En la siguiente gráfica interactiva, haz variar los parámetros estructurales del dosel (el LAI permanecerá fijo según la [serie temporal anterior](./input/canopy/LAI_FAPAR_ES-Cnd.csv)). Además puedes variar el nivel de estrés hídrico del dosel.

In [ ]:
w_ts2 = interactive(fn.simulate_flux_timeseries, {"manual": True, "manual_name": "Simular ET"},
                    h_c=fn.w_hc, f_c=fn.w_fc, w_c=fn.w_wc, hb_ratio=fn.w_hb_ratio, 
                    h_c_max=fn.w_h_c_max, leaf_angle=fn.w_leaf_angle,
                    stress=fn.w_stress, r_ss=fn.w_r_ss, date_range=fn.w_dates)
display(w_ts2)

## Estimación de ET con proximal LST usando TSEB
Basado en el model de dos capas de Shuttleworth & Wallace previamente visto, es posible estimar la ET (o flujo de calor latente $\lambda E$ mediantes observaciones de temperatura de superficie, bien tomadas a partir de satélites, vehículos aéreso como droes, o radiómetros térmicos en tierra (proximal LST). Para ello haremos uso del modelo de balance enegético de dos capas (TSEB en sus siglas en inglés).

A partir de los datos meteorológicos instantáneos (irradianca, velocidad del viento y temperatura del aire) y de temperatura de superfice (estimada con un radiómetro térmico que observa una mezcla de copa y suelo desnudo) TSEB realiza un proceso iterativo para hallar las temperaturas de copa y suelo. Además incorpora todos los procesos previamente descritos, de una manera eficiente, de modo que sea operativo para hacer un seguimiento en tiempo casi-real del consumo de agua y estrés hídrico de la vegetación. 

> Puedes consultar el código de TSEB [aquí](https://github.com/hectornieto/pyTSEB/blob/9bb28da739090d36f9cc0c1a771efb897c05eb48/pyTSEB/TSEB.py#L479)

En la siguiente tarea vamos a estimar la ET y compararla con los datos de la torre de flujos del olivar. Para ello puedes cambiar los parámetros estructurales del dosel a tu antojo y ver el efecto (o no) que éstos tienen en la estimación de la ET. 

> Los datos estructurales medidos en el olivar son:
> 
> * Altura del dosel, $h_c \approx 4$ m.
>
> * Fracción de cobertura, $f_c \approx 0.3$ m.
>
> * Relación ancho/alto de copa,  $Forma \approx 1$ m.
>
> * Altura relativa del inicio de la copa, $h_{bottom} \approx 0.3$.
>
> * Altura relativa donde se encuentra el máximo de biomasa foliar, $h_{max} \approx 0.5$.
>
> * Ángulo cenital predominante de la hojas, $LIDF \approx 57$ º.



In [ ]:
w_tseb = interactive(fn.run_tseb, {"manual": True, "manual_name": "Estimar ET"},
                     h_c=fn.w_hc, f_c=fn.w_fc, w_c=fn.w_wc, hb_ratio=fn.w_hb_ratio, 
                     h_c_max=fn.w_h_c_max, leaf_angle=fn.w_leaf_angle,
                     date_range=fn.w_dates)
display(w_tseb)

Y aquí puedes ver la serie temporal de estimaciones de ET

In [ ]:
if not isinstance(w_tseb.result, type(None)):
    w_ts = interactive(fn.create_flux_timeseries,
                       date_range=fn.w_dates, 
                       df=fixed(w_tseb.result[0]), daily_df=fixed(w_tseb.result[1]), 
                       include_obs=fixed(True))
    display(w_ts)

# Conclusiones
En esta práctica hemos visto que la radiación neta de onda corta depende principalmente de:
1. la radiacíón incidente, la cual puede obtenerse a partir de estaciones o modelos meteorológicos
2. El albedo de la superficie

Por otro lado el albedo depende principalmente de:
1. Las propiedades espectrales de la vegetación, principalmente la clorofila como principal pigmento absorbente de radiación PAR.
2. **Las propiedades estructurales de la vegetación**, principalmente el LAI pero también de la distribución angular de la hoja. Además en cultivos heterogéneos la arquitectura del dosel puede resultar determinante en el reparto de radiaicón entre suelo y vegetación.

Los intercambios de calor y agua entre el suelo, la vegetación y la atmósfera dependen de:
1. El contenido de humedad en la capa superficial del suelo, que determina la evaporación directa del suelo.
2. El contenido de humedad en la zona radicular del suelo, que determina la transpiración de la planta y su capacidad fotosíntetica.
3. **La estructura del dosel**, el cual determina cómo de eficiente se disipa hacia la atmósfera el vapor de agua y el calor generado.

La temperatura radiométrica de la superficie es un factor clave a la hora de evaluar el estŕes hídrico y cuantificar el consumo de agua (o evapotranspiración) de las plantas. 

Existen un gran abanico de modelos que permiten estimar la ET a partir de datos de teledetección. En esta práctica hemos usado el modelo TSEB, ya que ha demostrado una gran robusted para esimar la ET en un amplio espectro de tipos de vegetación y climas. Además el modelo y todos los submodelos que requiere para evaluar los flujos de calor, agua y carbono está disponible en código abierto [aquí](https://github.com/hectornieto/pyTSEB). 
